### LSTM with Bahdanau attention

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class BahdanauAttention(nn.Module):
    """
    Механизм внимания Бахданау (Additive Attention).
    Позволяет декодеру фокусироваться на разных частях выхода энкодера.
    """
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim, attention_dim):
        """
        Args:
            encoder_hidden_dim (int): Размер скрытого состояния энкодера (H_enc).
            decoder_hidden_dim (int): Размер скрытого состояния декодера (H_dec).
            attention_dim (int): Размер внутреннего представления внимания.
        """
        super(BahdanauAttention, self).__init__()
        self.encoder_hidden_dim = encoder_hidden_dim
        self.decoder_hidden_dim = decoder_hidden_dim
        self.attention_dim = attention_dim

        # Линейные слои для преобразования входов перед вычислением внимания
        self.W_enc = nn.Linear(encoder_hidden_dim, attention_dim, bias=False)
        self.W_dec = nn.Linear(decoder_hidden_dim, attention_dim, bias=False)
        self.V = nn.Linear(attention_dim, 1, bias=False)

        # Инициализация весов для лучшей сходимости
        self._init_weights()


    def _init_weights(self):
        """Инициализация весов для лучшей сходимости."""
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_uniform_(param)


    def forward(self, encoder_outputs, decoder_hidden):
        """
        Вычисляет веса внимания и контекстный вектор.
        
        Args:
            encoder_outputs (torch.Tensor): Выходы энкодера [B, T_hist, H_enc].
            decoder_hidden (torch.Tensor): Скрытое состояние декодера [B, H_dec].
            
        Returns:
            context_vector (torch.Tensor): Контекстный вектор [B, H_enc].
            attention_weights (torch.Tensor): Веса внимания [B, T_hist].
        """
        batch_size, seq_len, _ = encoder_outputs.size()
        
        # decoder_hidden: [B, H_dec] -> [B, 1, H_dec] для broadcast
        decoder_hidden_expanded = decoder_hidden.unsqueeze(1)
        
        # Вычисляем оценку внимания e_tj
        energy = torch.tanh(
            self.W_enc(encoder_outputs) + self.W_dec(decoder_hidden_expanded)
        )
        attention_scores = self.V(energy).squeeze(2) # [B, T_hist]

        # Применяем softmax для получения весов внимания
        attention_weights = F.softmax(attention_scores, dim=1) # [B, T_hist]

        # Вычисляем контекстный вектор как взвешенную сумму выходов энкодера
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        
        return context_vector, attention_weights



class LSTMEncoder(nn.Module):
    """
    LSTM Энкодер для обработки исторических данных.
    """
    def __init__(self, input_size, hidden_size, num_layers, dropout=0.0, bidirectional=False):
        """
        Args:
            input_size (int): Размер входных признаков (128).
            hidden_size (int): Размер скрытого состояния LSTM.
            num_layers (int): Количество слоев LSTM.
            dropout (float): Вероятность Dropout между слоями.
            bidirectional (bool): Использовать_bidirectional LSTM.
        """
        super(LSTMEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.num_directions = 2 if bidirectional else 1
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0,
            bidirectional=bidirectional
        )
        
        # Если_bidirectional, добавляем слой для проекции выходов в нужный размер
        if bidirectional:
            self.output_projection = nn.Linear(hidden_size * 2, hidden_size)
        else:
            self.output_projection = None
            
        self._init_weights()


    def _init_weights(self):
        """Инициализация весов."""
        for name, param in self.named_parameters():
            if 'weight' in name and len(param.shape) > 1:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)


    def forward(self, x):
        """
        Прогоняет вход через LSTM.
        
        Args:
            x (torch.Tensor): Входные данные [B, T_hist, input_size].
            
        Returns:
            outputs (torch.Tensor): Все скрытые состояния [B, T_hist, hidden_size * num_directions].
            (h_n, c_n) (tuple): Финальные скрытое и ячейковое состояния [num_layers * num_directions, B, hidden_size].
        """
        outputs, (h_n, c_n) = self.lstm(x)
        
        # Если_bidirectional, проецируем выходы
        if self.bidirectional:
            outputs = self.output_projection(outputs)
            
        return outputs, (h_n, c_n)



class LSTMDecoder(nn.Module):
    """
    LSTM Декодер с механизмом внимания для генерации прогноза.
    """
    def __init__(self, input_size, hidden_size, num_layers, output_size, attention, dropout=0.0):
        """
        Args:
            input_size (int): Размер входных признаков для декодера (обычно 1 - цена закрытия предыдущего шага).
            hidden_size (int): Размер скрытого состояния LSTM декодера.
            num_layers (int): Количество слоев LSTM декодера.
            output_size (int): Размер выхода на каждом шаге (обычно 1).
            attention (BahdanauAttention): Экземпляр механизма внимания.
            dropout (float): Вероятность Dropout.
        """
        super(LSTMDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        self.attention = attention
        
        # Вход декодера состоит из:
        # 1. Предыдущий прогноз/таргет (input_size)
        # 2. Контекстный вектор от внимания (attention.encoder_hidden_dim)
        self.lstm = nn.LSTM(
            input_size=input_size + attention.encoder_hidden_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0.0
        )
        
        # Полносвязный слой для преобразования выхода LSTM декодера в размер прогноза
        self.fc = nn.Sequential(
            nn.Linear(hidden_size + attention.encoder_hidden_dim, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, output_size)
        )
        
        self._init_weights()


    def _init_weights(self):
        """Инициализация весов."""
        for name, param in self.named_parameters():
            if 'weight' in name and len(param.shape) > 1:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)


    def forward(self, target_len, encoder_outputs, encoder_hidden, encoder_cell, teacher_forcing_ratio=0.5, targets=None, initial_input=None):
        """
        Генерирует последовательность прогнозов.
        
        Args:
            target_len (int): Длина последовательности для генерации (32).
            encoder_outputs (torch.Tensor): Выходы энкодера [B, T_hist, H_enc].
            encoder_hidden (torch.Tensor): Финальное скрытое состояние энкодера [num_layers_enc * num_directions, B, H_enc].
            encoder_cell (torch.Tensor): Финальное ячейковое состояние энкодера [num_layers_enc * num_directions, B, H_enc].
            teacher_forcing_ratio (float): Вероятность использования teacher forcing.
            targets (torch.Tensor, optional): Реальные таргеты [B, T_pred, 1] для teacher forcing.
            initial_input (torch.Tensor, optional): Начальное значение для декодера [B, 1, input_size].
            
        Returns:
            outputs (torch.Tensor): Сгенерированные прогнозы [B, T_pred, output_size].
            attention_weights_list (list): Список весов внимания для каждого шага [T_pred, B, T_hist].
        """
        batch_size = encoder_outputs.size(0)
        device = encoder_outputs.device
        
        # Инициализируем выходы декодера
        outputs = torch.zeros(batch_size, target_len, self.output_size, device=device)
        
        # Инициализируем список для хранения весов внимания
        attention_weights_list = []
        
        # Инициализируем вход декодера
        if initial_input is not None:
            decoder_input = initial_input # [B, 1, input_size]
        else:
            # Используем ноль как начальное значение
            decoder_input = torch.zeros(batch_size, 1, 1, device=device) # [B, 1, 1]

        # Инициализируем скрытое и ячейковое состояние декодера
        # Берем последние num_layers слоев из encoder_hidden/encoder_cell
        decoder_hidden = encoder_hidden[-self.num_layers:] if encoder_hidden.size(0) >= self.num_layers else encoder_hidden
        decoder_cell = encoder_cell[-self.num_layers:] if encoder_cell.size(0) >= self.num_layers else encoder_cell
        
        for t in range(target_len):
            # Вычисляем контекстный вектор с помощью внимания
            context_vector, attention_weights = self.attention(encoder_outputs, decoder_hidden[-1])
            attention_weights_list.append(attention_weights)
            
            # Подготовка входа для LSTM декодера
            lstm_input = torch.cat((decoder_input, context_vector.unsqueeze(1)), dim=2)
            
            # Прогоняем через LSTM
            lstm_output, (decoder_hidden, decoder_cell) = self.lstm(lstm_input, (decoder_hidden, decoder_cell))
            
            # Подготовка входа для полносвязного слоя
            fc_input = torch.cat((lstm_output.squeeze(1), context_vector), dim=1)
            
            # Прогноз на текущем шаге
            output = self.fc(fc_input)
            outputs[:, t:t+1] = output.unsqueeze(1)
            
            # Подготовка decoder_input для следующего шага
            use_teacher_forcing = torch.rand(1).item() < teacher_forcing_ratio
            if use_teacher_forcing and targets is not None and t < targets.size(1):
                decoder_input = targets[:, t:t+1]
            else:
                decoder_input = output.unsqueeze(1)

        return outputs, attention_weights_list



class TradingLSTM(nn.Module):
    """
    Полная LSTM Encoder-Decoder модель с вниманием для прогнозирования цен.
    """
    def __init__(
        self,
        feature_size=128,
        encoder_hidden_size=256,
        encoder_num_layers=2,
        decoder_hidden_size=256,
        decoder_num_layers=2,
        attention_dim=128,
        target_len=32,
        dropout=0.2,
        teacher_forcing_ratio=0.5,
        bidirectional_encoder=True,
        use_layer_norm=True
    ):
        """
        Args:
            feature_size (int): Размер входных признаков после TradingProcessor (128).
            encoder_hidden_size (int): Размер скрытого состояния энкодера.
            encoder_num_layers (int): Количество слоев энкодера.
            decoder_hidden_size (int): Размер скрытого состояния декодера.
            decoder_num_layers (int): Количество слоев декодера.
            attention_dim (int): Размер внутреннего представления внимания.
            target_len (int): Длина прогнозируемой последовательности (32).
            dropout (float): Вероятность Dropout.
            teacher_forcing_ratio (float): Вероятность использования teacher forcing при обучении.
            bidirectional_encoder (bool): Использовать_bidirectional LSTM в энкодере.
            use_layer_norm (bool): Использовать Layer Normalization.
        """
        super(TradingLSTM, self).__init__()
        self.target_len = target_len
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.bidirectional_encoder = bidirectional_encoder
        
        # Создаем механизм внимания
        self.attention = BahdanauAttention(
            encoder_hidden_dim=encoder_hidden_size,
            decoder_hidden_dim=decoder_hidden_size,
            attention_dim=attention_dim
        )
        
        # Создаем энкодер
        self.encoder = LSTMEncoder(
            input_size=feature_size,
            hidden_size=encoder_hidden_size,
            num_layers=encoder_num_layers,
            dropout=dropout,
            bidirectional=bidirectional_encoder
        )
        
        # Создаем декодер
        self.decoder = LSTMDecoder(
            input_size=1, # Прогнозируемая цена закрытия
            hidden_size=decoder_hidden_size,
            num_layers=decoder_num_layers,
            output_size=1, # Одна цена закрытия на выходе
            attention=self.attention,
            dropout=dropout
        )
        
        # Дополнительный слой нормализации (опционально)
        self.use_layer_norm = use_layer_norm
        if use_layer_norm:
            self.layer_norm = nn.LayerNorm(feature_size)
            
        self._init_weights()


    def _init_weights(self):
        """Инициализация весов."""
        for name, param in self.named_parameters():
            if 'weight' in name and len(param.shape) > 1:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)


    def forward(self, src, tgt=None):
        """
        Прямой проход модели.
        
        Args:
            src (torch.Tensor): Исторические данные после обработки [B, T_hist, F_hist=128].
            tgt (torch.Tensor, optional): Целевые значения (таргеты) [B, T_pred, 1]. Используется для teacher forcing.
            
        Returns:
            output (torch.Tensor): Прогнозы [B, T_pred, 1].
        """
        batch_size = src.size(0)
        
        # Применяем Layer Normalization (если включено)
        if self.use_layer_norm:
            src = self.layer_norm(src)
        
        # 1. Энкодер
        encoder_outputs, (encoder_hidden, encoder_cell) = self.encoder(src)
        
        # 2. Подготовка начального значения для декодера
        # Можно использовать последнее значение цены закрытия из исходных данных
        # Предполагаем, что цена закрытия - это 4-й столбец в исходных данных (до обработки)
        # Но так как у нас уже обработанные данные, можно использовать среднее или другое значение
        # Для простоты возьмем среднее значение последних N точек
        # initial_input = src[:, -1:, 3:4] # Если бы у нас были исходные данные
        # Или просто ноль
        initial_input = None # Пусть декодер сам решит
        
        # 3. Декодер
        decoder_outputs, _ = self.decoder(
            target_len=self.target_len,
            encoder_outputs=encoder_outputs,
            encoder_hidden=encoder_hidden,
            encoder_cell=encoder_cell,
            teacher_forcing_ratio=self.teacher_forcing_ratio if self.training else 0.0,
            targets=tgt,
            initial_input=initial_input
        )
        
        return decoder_outputs



# --- Пример использования ---
if __name__ == "__main__":
    # Параметры модели
    B, T_hist, feature_size = 4, 256, 128
    T_pred = 32
    output_size = 1
    
    # Создание модели
    model = TradingLSTM(
        feature_size=feature_size,
        encoder_hidden_size=512,
        encoder_num_layers=3,
        decoder_hidden_size=512,
        decoder_num_layers=3,
        attention_dim=128,
        target_len=T_pred,
        dropout=0.2,
        teacher_forcing_ratio=0.5,
        bidirectional_encoder=True, # Новое улучшение
        use_layer_norm=True # Новое улучшение
    )
    
    # Примерные входные данные
    src = torch.randn(B, T_hist, feature_size)  # История после TradingProcessor
    tgt = torch.randn(B, T_pred, output_size)   # Целевые значения
    
    # Обучение (с учителем)
    model.train()
    output_train = model(src, tgt)
    print(f"Выход при обучении (с учителем): {output_train.shape}") # [B, 32, 1]
    
    # Инференс
    model.eval()
    with torch.no_grad():
        output_infer = model(src)
        print(f"Выход при инференсе: {output_infer.shape}") # [B, 32, 1]


Выход при обучении (с учителем): torch.Size([4, 32, 1])
Выход при инференсе: torch.Size([4, 32, 1])


### Temporal Convolutional Network (TCN)

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Chomp1d(nn.Module):
    """
    Отсекает лишние элементы из последовательности после свертки с паддингом.
    Обеспечивает причинность (causality) свертки.
    """
    def __init__(self, chomp_size):
        """
        Args:
            chomp_size (int): Количество элементов для отсечения с правого края.
        """
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size


    def forward(self, x):
        """
        Args:
            x (Tensor): Входной тензор [B, C, T]
        Returns:
            Tensor: Выходной тензор [B, C, T - chomp_size]
        """
        return x[:, :, :-self.chomp_size].contiguous()



class TemporalBlock(nn.Module):
    """
    Базовый блок TCN: две причинные свертки с Dilated Conv + ReLU + Dropout.
    Использует остаточные соединения.
    """
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        """
        Args:
            n_inputs (int): Количество входных каналов.
            n_outputs (int): Количество выходных каналов.
            kernel_size (int): Размер ядра свертки.
            stride (int): Шаг свертки.
            dilation (int): Коэффициент расширения (dilation).
            padding (int): Размер паддинга.
            dropout (float): Вероятность Dropout.
        """
        super(TemporalBlock, self).__init__()
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.kernel_size = kernel_size
        self.dilation = dilation
        
        # Первый сверточный слой
        self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)  # Убираем паддинг, чтобы сохранить причинность
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        # Второй сверточный слой
        self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size,
                               stride=stride, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        # Последовательность для первого блока
        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)

        # Слой проекции для остаточного соединения, если размерности не совпадают
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        
        self.init_weights()


    def init_weights(self):
        """Инициализация весов сверточных слоев."""
        # Инициализация для conv1
        nn.init.kaiming_normal_(self.conv1.weight, mode='fan_out', nonlinearity='relu')
        if self.conv1.bias is not None:
            nn.init.constant_(self.conv1.bias, 0)
            
        # Инициализация для conv2
        nn.init.kaiming_normal_(self.conv2.weight, mode='fan_out', nonlinearity='relu')
        if self.conv2.bias is not None:
            nn.init.constant_(self.conv2.bias, 0)
            
        # Инициализация слоя проекции (если он есть)
        if self.downsample is not None:
            nn.init.kaiming_normal_(self.downsample.weight, mode='fan_out', nonlinearity='relu')
            if self.downsample.bias is not None:
                nn.init.constant_(self.downsample.bias, 0)


    def forward(self, x):
        """
        Прямой проход через блок.
        Args:
            x (Tensor): Входной тензор [B, n_inputs, T]
        Returns:
            Tensor: Выходной тензор [B, n_outputs, T]
        """
        out = self.net(x)
        # Остаточное соединение
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)



class TemporalConvNet(nn.Module):
    """
    Полная TCN архитектура, состоящая из стека TemporalBlock'ов.
    """
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        """
        Args:
            num_inputs (int): Количество входных признаков.
            num_channels (list): Список количества каналов для каждого блока.
                             Например, [256, 256, 256] означает 3 блока по 256 каналов.
            kernel_size (int): Размер ядра свертки.
            dropout (float): Вероятность Dropout.
        """
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        
        for i in range(num_levels):
            # Вычисляем параметры для текущего блока
            dilation_size = 2 ** i  # Экспоненциально увеличиваем dilation
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            
            # Добавляем блок
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, 
                                   dilation=dilation_size, padding=(kernel_size-1) * dilation_size, 
                                   dropout=dropout)]

        self.network = nn.Sequential(*layers)
        

    def forward(self, x):
        """
        Прямой проход через всю сеть.
        Args:
            x (Tensor): Входной тензор [B, num_inputs, T]
        Returns:
            Tensor: Выходной тензор [B, num_channels[-1], T]
        """
        return self.network(x)



class TradingTCN(nn.Module):
    """
    Полная модель TCN для прогнозирования цен на фондовом рынке.
    """
    def __init__(
        self,
        feature_size=128,           # Размер признаков от TradingProcessor
        num_channels=[64, 64, 64, 128, 128, 128, 256, 256, 512, 512], # Архитектура TCN
        kernel_size=3,              # Размер ядра свертки
        dropout=0.2,                # Dropout
        target_len=32,              # Длина прогноза
        use_layer_norm=True,        # Использовать LayerNorm
        use_weight_norm=True        # Использовать WeightNorm
    ):
        """
        Args:
            feature_size (int): Размер входных признаков (128).
            num_channels (list): Список количества каналов для каждого TCN блока.
            kernel_size (int): Размер ядра свертки.
            dropout (float): Вероятность Dropout.
            target_len (int): Длина прогнозируемой последовательности (32).
            use_layer_norm (bool): Применять ли LayerNorm к входу.
            use_weight_norm (bool): Применять ли WeightNorm к сверточным слоям.
        """
        super(TradingTCN, self).__init__()
        self.feature_size = feature_size
        self.target_len = target_len
        self.use_layer_norm = use_layer_norm
        self.use_weight_norm = use_weight_norm
        
        # LayerNorm для нормализации входа
        if use_layer_norm:
            self.layer_norm = nn.LayerNorm(feature_size)
        
        # TCN сеть
        self.tcn = TemporalConvNet(feature_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        
        # Выходной слой для прогнозирования
        # Вход: [B, num_channels[-1], T_hist]
        # Выход: [B, 1, T_hist] (прогнозируем только цену закрытия)
        self.output_projection = nn.Conv1d(num_channels[-1], 1, 1) # 1x1 conv для проекции каналов
        
        # Дополнительный слой для генерации последовательности прогноза
        # Мы можем использовать последние target_len точек из выхода TCN
        # или применить дополнительную обработку
        
        self.init_weights()
        
        # Применяем WeightNorm, если требуется
        if use_weight_norm:
            self.apply_weight_norm()
            

    def apply_weight_norm(self):
        """Применяет WeightNorm к сверточным слоям."""
        for module in self.modules():
            if isinstance(module, (nn.Conv1d, nn.ConvTranspose1d)):
                module = nn.utils.weight_norm(module)


    def init_weights(self):
        """Инициализация весов выходного слоя."""
        nn.init.kaiming_normal_(self.output_projection.weight, mode='fan_out', nonlinearity='relu')
        if self.output_projection.bias is not None:
            nn.init.constant_(self.output_projection.bias, 0)


    def forward(self, src, tgt=None):
        """
        Прямой проход модели.
        
        Args:
            src (torch.Tensor): Исторические данные [B, T_hist, F_hist=128].
                                Должны быть уже обработаны TradingProcessor.
            tgt (torch.Tensor, optional): Целевые значения [B, T_pred, 1].
            
        Returns:
            torch.Tensor: Прогнозы [B, T_pred, 1].
        """
        # src: [B, T_hist, 128]
        batch_size, seq_len, _ = src.size()
        
        # Применяем Layer Normalization (если включено)
        if self.use_layer_norm:
            src = self.layer_norm(src) # [B, T_hist, 128]
            
        # Переставляем размерности для TCN: [B, F_hist, T_hist]
        x = src.transpose(1, 2).contiguous() # [B, 128, T_hist]
        
        # Пропускаем через TCN
        # y: [B, num_channels[-1], T_hist]
        y = self.tcn(x)
        
        # Применяем выходную проекцию для получения прогноза цены закрытия
        # output: [B, 1, T_hist]
        output = self.output_projection(y)
        
        # Берем последние target_len точек для прогноза
        # output: [B, 1, T_hist] -> [B, 1, target_len] -> [B, target_len, 1]
        prediction = output[:, :, -self.target_len:].transpose(1, 2)
        
        return prediction



# --- Пример использования ---
if __name__ == "__main__":
    # Параметры модели
    B, T_hist, feature_size = 4, 256, 128
    T_pred = 32
    output_size = 1
    
    # Создание модели
    model = TradingTCN(
        feature_size=feature_size,
        num_channels=[64, 64, 128, 128, 256, 256, 512, 512], # Глубокая архитектура
        kernel_size=3,
        dropout=0.2,
        target_len=T_pred,
        use_layer_norm=True,
        use_weight_norm=True
    )
    
    # Примерные входные данные
    src = torch.randn(B, T_hist, feature_size)  # История после TradingProcessor
    tgt = torch.randn(B, T_pred, output_size)   # Целевые значения (опционально)
    
    # Прогон модели
    model.eval()
    with torch.no_grad():
        output = model(src, tgt)
        print(f"Выход модели: {output.shape}") # [B, 32, 1]


Выход модели: torch.Size([4, 32, 1])


### CNN

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BottleneckBlock(nn.Module):
    """
    Bottleneck блок ResNet, как в ResNet-50/101/152.
    Состоит из 1x1 -> 3x3 -> 1x1 сверток.
    """
    expansion = 4  # Коэффициент увеличения размера выхода по каналам

    def __init__(self, in_channels, out_channels, stride=1, downsample=None, dropout=0.1):
        """
        Args:
            in_channels (int): Количество входных каналов.
            out_channels (int): Количество выходных каналов (до expansion).
            stride (int): Шаг свертки (используется в 3x3 свертке для downsampling).
            downsample (nn.Module, optional): Слой для проекции shortcut, если необходимо.
            dropout (float): Вероятность Dropout.
        """
        super(BottleneckBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, 
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        # Третья свертка увеличивает количество каналов в expansion раз
        self.conv3 = nn.Conv1d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm1d(out_channels * self.expansion)
        
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Args:
            x (Tensor): Входной тензор [B, C_in, T]
        Returns:
            Tensor: Выходной тензор [B, C_out_expanded, T]
        """
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
        
        # Применяем downsampling к shortcut, если необходимо
        if self.downsample is not None:
            identity = self.downsample(x)

        # Остаточное соединение
        out += identity
        out = self.relu(out)
        out = self.dropout(out)

        return out


class ResNetCNN(nn.Module):
    """
    ResNet-подобная CNN для обработки временных рядов.
    Адаптирована для 1D данных (временные ряды).
    """
    def __init__(self, block, layers, input_channels=128, num_classes=32, dropout=0.2):
        """
        Args:
            block (nn.Module): Тип блока (BottleneckBlock).
            layers (list): Список, определяющий количество блоков в каждом слое.
                          Например, [3, 4, 23, 3] для ResNet-101.
            input_channels (int): Количество входных признаков (128 от TradingProcessor).
            num_classes (int): Размерность выхода (32 точки прогноза).
            dropout (float): Вероятность Dropout в голове.
        """
        super(ResNetCNN, self).__init__()
        self.in_channels = 64  # Начальное количество каналов
        self.dropout = dropout

        # Начальный слой для преобразования входа в последовательность признаков
        # Предполагаем, что вход [B, T, F] -> [B, F, T] для Conv1d
        # Но мы хотим обрабатывать временные зависимости, поэтому свертка по времени
        # Для этого вход будет [B, F_in, T], где F_in=128, T=256
        # Начальный conv1d преобразует F_in -> 64 признаков
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        # Создаем слои ResNet
        # Для 1D данные, "layer1" не делает downsampling (stride=1)
        self.layer1 = self._make_layer(block, 64, layers[0], dropout=dropout)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dropout=dropout)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dropout=dropout)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dropout=dropout)

        self.avgpool = nn.AdaptiveAvgPool1d(1) # Глобальный average pooling
        
        # Голова для прогнозирования
        # После layer4 у нас будет 512 * expansion = 512 * 4 = 2048 каналов
        self.fc_head = nn.Sequential(
            nn.Linear(512 * block.expansion, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes) # Выход [B, 32]
        )
        
        # Инициализация весов
        self._init_weights()

    def _make_layer(self, block, out_channels, blocks, stride=1, dropout=0.1):
        """
        Создает слой ResNet, состоящий из нескольких bottleneck блоков.
        """
        downsample = None
        # Если stride != 1 или количество каналов изменилось, нужна проекция shortcut
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.in_channels, out_channels * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels * block.expansion),
            )

        layers = []
        # Первый блок может делать downsampling
        layers.append(block(self.in_channels, out_channels, stride, downsample, dropout))
        self.in_channels = out_channels * block.expansion
        # Остальные блоки без downsampling
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels, dropout=dropout))

        return nn.Sequential(*layers)

    def _init_weights(self):
        """Инициализация весов."""
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        """
        Прямой проход.
        Args:
            x (Tensor): Входной тензор [B, F_in=128, T=256]
        Returns:
            Tensor: Выходной тензор [B, num_classes=32]
        """
        # x: [B, 128, 256]
        x = self.conv1(x)       # [B, 64, 128]
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)     # [B, 64, 64]

        x = self.layer1(x)      # [B, 256, 64]  (64 = 64 * 4)
        x = self.layer2(x)      # [B, 512, 32]  (512 = 128 * 4)
        x = self.layer3(x)      # [B, 1024, 16] (1024 = 256 * 4)
        x = self.layer4(x)      # [B, 2048, 8]  (2048 = 512 * 4)

        # Глобальный average pooling: [B, 2048, 8] -> [B, 2048, 1] -> [B, 2048]
        x = self.avgpool(x).view(x.size(0), -1)
        
        # Голова прогнозирования: [B, 2048] -> [B, 32]
        x = self.fc_head(x)
        
        # Добавляем размерность для совместимости с другими моделями [B, 32] -> [B, 32, 1]
        x = x.unsqueeze(-1)
        
        return x


class TradingCNN(nn.Module):
    """
    Мощная CNN модель на основе ResNet-101 для прогнозирования цен на фондовом рынке.
    """
    def __init__(
        self,
        feature_size=128,           # Размер признаков от TradingProcessor
        target_len=32,              # Длина прогноза
        dropout=0.2,                # Dropout
        use_layer_norm=True,        # Использовать LayerNorm на входе
        resnet_layers=[3, 4, 23, 3] # Конфигурация слоев ResNet (ResNet-101)
    ):
        """
        Args:
            feature_size (int): Размер входных признаков (128).
            target_len (int): Длина прогнозируемой последовательности (32).
            dropout (float): Вероятность Dropout.
            use_layer_norm (bool): Применять ли LayerNorm к входу.
            resnet_layers (list): Конфигурация слоев ResNet.
        """
        super(TradingCNN, self).__init__()
        self.feature_size = feature_size
        self.target_len = target_len
        self.use_layer_norm = use_layer_norm
        
        # LayerNorm для нормализации входа
        if use_layer_norm:
            self.layer_norm = nn.LayerNorm(feature_size)
            
        # Основная ResNet-подобная CNN
        self.resnet_cnn = ResNetCNN(
            block=BottleneckBlock,
            layers=resnet_layers, # [3, 4, 23, 3] для ResNet-101
            input_channels=feature_size,
            num_classes=target_len,
            dropout=dropout
        )

    def forward(self, src, tgt=None):
        """
        Прямой проход модели.
        
        Args:
            src (torch.Tensor): Исторические данные [B, T_hist=256, F_hist=128].
                                Должны быть уже обработаны TradingProcessor.
            tgt (torch.Tensor, optional): Целевые значения [B, T_pred, 1].
            
        Returns:
            torch.Tensor: Прогнозы [B, T_pred=32, 1].
        """
        # src: [B, 256, 128]
        batch_size, seq_len, _ = src.size()
        
        # Применяем Layer Normalization (если включено)
        if self.use_layer_norm:
            src = self.layer_norm(src) # [B, 256, 128]
            
        # Переставляем размерности для CNN: [B, F_hist, T_hist]
        x = src.transpose(1, 2).contiguous() # [B, 128, 256]
        
        # Пропускаем через ResNet CNN
        # y: [B, 32, 1]
        y = self.resnet_cnn(x)
        
        return y
    


# --- Пример использования ---
if __name__ == "__main__":
    # Параметры модели
    B, T_hist, feature_size = 4, 256, 128
    T_pred = 32
    output_size = 1
    
    # Создание модели (ResNet-101)
    model = TradingCNN(
        feature_size=feature_size,
        target_len=T_pred,
        dropout=0.2,
        use_layer_norm=True,
        resnet_layers=[3, 4, 23, 3] # ResNet-101
    ).to('mps')
    
    # Примерные входные данные
    src = torch.randn(B, T_hist, feature_size).to('mps')  # История после TradingProcessor
    tgt = torch.randn(B, T_pred, output_size).to('mps')   # Целевые значения (опционально)
    
    # Прогон модели
    model.eval()
    with torch.no_grad():
        output = model(src, tgt)
        print(f"Выход модели: {output.shape}") # [B, 32, 1]

Выход модели: torch.Size([4, 32, 1])


### Transformer

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    """
    Позиционное кодирование для добавления информации о позиции в последовательности.
    """
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        """
        Args:
            d_model (int): Размерность модели (признаков).
            max_len (int): Максимальная длина последовательности.
            dropout (float): Вероятность Dropout.
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Создаем матрицу позиционных кодировок: [max_len, d_model]
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # [max_len, 1]
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        
        # Добавляем размерность для батча: [1, max_len, d_model]
        pe = pe.unsqueeze(0)
        # Регистрируем как буфер, чтобы он сохранялся при сериализации модели
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x (Tensor): Входной тензор [B, seq_len, d_model]
        Returns:
            Tensor: Выходной тензор [B, seq_len, d_model]
        """
        # x: [B, seq_len, d_model]
        # self.pe: [1, max_len, d_model]
        x = x + self.pe[:, :x.size(1)] # Broadcasting по батчу
        return self.dropout(x)


class MultiHeadAttention(nn.Module):
    """
    Многоголовое внимание (Multi-Head Attention).
    """
    def __init__(self, d_model, num_heads, dropout=0.1):
        """
        Args:
            d_model (int): Размерность модели.
            num_heads (int): Количество голов внимания.
            dropout (float): Вероятность Dropout.
        """
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads # Размерность ключа/запроса/значения на одну голову

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)
        self.scale = math.sqrt(self.d_k)

    def forward(self, query, key, value, mask=None):
        """
        Args:
            query (Tensor): Запросы [B, len_q, d_model]
            key (Tensor): Ключи [B, len_k, d_model]
            value (Tensor): Значения [B, len_v, d_model]
            mask (Tensor, optional): Маска [B, 1, len_q, len_k] или [B, num_heads, len_q, len_k]
        Returns:
            Tensor: Выход [B, len_q, d_model]
            Tensor: Веса внимания [B, num_heads, len_q, len_k]
        """
        batch_size = query.size(0)

        # Линейные преобразования и разделение на головы
        # [B, len, d_model] -> [B, len, num_heads, d_k] -> [B, num_heads, len, d_k]
        Q = self.W_q(query).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # Матрица оценок внимания
        # [B, num_heads, len_q, d_k] x [B, num_heads, d_k, len_k] -> [B, num_heads, len_q, len_k]
        scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scale

        # Применение маски (если есть)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        # Вычисление весов внимания
        attn_weights = F.softmax(scores, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Применение весов к значениям
        # [B, num_heads, len_q, len_k] x [B, num_heads, len_v, d_k] -> [B, num_heads, len_q, d_k]
        # len_k == len_v
        attn_output = torch.matmul(attn_weights, V)

        # Конкатенация голов и линейное преобразование
        # [B, num_heads, len_q, d_k] -> [B, len_q, num_heads, d_k] -> [B, len_q, d_model]
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        output = self.W_o(attn_output)

        return output, attn_weights


class PositionwiseFeedForward(nn.Module):
    """
    Позиционно-независимый полносвязный слой (Feed-Forward Network).
    """
    def __init__(self, d_model, d_ff, dropout=0.1):
        """
        Args:
            d_model (int): Размерность модели.
            d_ff (int): Размерность внутреннего слоя.
            dropout (float): Вероятность Dropout.
        """
        super(PositionwiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Args:
            x (Tensor): Входной тензор [B, seq_len, d_model]
        Returns:
            Tensor: Выходной тензор [B, seq_len, d_model]
        """
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


class EncoderBlock(nn.Module):
    """
    Блок энкодера: Multi-Head Attention + Feed Forward.
    """
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        Args:
            d_model (int): Размерность модели.
            num_heads (int): Количество голов внимания.
            d_ff (int): Размерность внутреннего слоя FFN.
            dropout (float): Вероятность Dropout.
        """
        super(EncoderBlock, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None):
        """
        Args:
            src (Tensor): Входные данные [B, seq_len, d_model]
            src_mask (Tensor, optional): Маска для внимания [B, 1, 1, seq_len] или подобная
        Returns:
            Tensor: Выходные данные [B, seq_len, d_model]
        """
        # Self-attention
        attn_out, _ = self.self_attn(src, src, src, src_mask)
        src = self.norm1(src + self.dropout1(attn_out))
        
        # Feed forward
        ffn_out = self.ffn(src)
        out = self.norm2(src + self.dropout2(ffn_out))
        
        return out


class DecoderBlock(nn.Module):
    """
    Блок декодера: Masked Multi-Head Attention + Multi-Head Attention + Feed Forward.
    """
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        """
        Args:
            d_model (int): Размерность модели.
            num_heads (int): Количество голов внимания.
            d_ff (int): Размерность внутреннего слоя FFN.
            dropout (float): Вероятность Dropout.
        """
        super(DecoderBlock, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.cross_attn = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
        """
        Args:
            tgt (Tensor): Вход декодера (таргеты) [B, tgt_len, d_model]
            memory (Tensor): Выход энкодера [B, src_len, d_model]
            tgt_mask (Tensor, optional): Маска для self-attention в декодере [B, 1, tgt_len, tgt_len]
            memory_mask (Tensor, optional): Маска для cross-attention [B, 1, 1, src_len]
        Returns:
            Tensor: Выход декодера [B, tgt_len, d_model]
        """
        # Masked self-attention (causal)
        self_attn_out, _ = self.self_attn(tgt, tgt, tgt, tgt_mask)
        tgt = self.norm1(tgt + self.dropout1(self_attn_out))
        
        # Cross-attention
        cross_attn_out, _ = self.cross_attn(tgt, memory, memory, memory_mask)
        tgt = self.norm2(tgt + self.dropout2(cross_attn_out))
        
        # Feed forward
        ffn_out = self.ffn(tgt)
        out = self.norm3(tgt + self.dropout3(ffn_out))
        
        return out


class TransformerEncoder(nn.Module):
    """
    Полный энкодер трансформера.
    """
    def __init__(self, num_layers, d_model, num_heads, d_ff, dropout=0.1):
        """
        Args:
            num_layers (int): Количество блоков энкодера.
            d_model (int): Размерность модели.
            num_heads (int): Количество голов внимания.
            d_ff (int): Размерность внутреннего слоя FFN.
            dropout (float): Вероятность Dropout.
        """
        super(TransformerEncoder, self).__init__()
        self.layers = nn.ModuleList([
            EncoderBlock(d_model, num_heads, d_ff, dropout) 
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, src, src_mask=None):
        """
        Args:
            src (Tensor): Входные данные [B, src_len, d_model]
            src_mask (Tensor, optional): Маска для внимания [B, 1, 1, src_len]
        Returns:
            Tensor: Выходные данные [B, src_len, d_model]
        """
        output = src
        for layer in self.layers:
            output = layer(output, src_mask)
        return self.norm(output)


class TransformerDecoder(nn.Module):
    """
    Полный декодер трансформера.
    """
    def __init__(self, num_layers, d_model, num_heads, d_ff, dropout=0.1):
        """
        Args:
            num_layers (int): Количество блоков декодера.
            d_model (int): Размерность модели.
            num_heads (int): Количество голов внимания.
            d_ff (int): Размерность внутреннего слоя FFN.
            dropout (float): Вероятность Dropout.
        """
        super(TransformerDecoder, self).__init__()
        self.layers = nn.ModuleList([
            DecoderBlock(d_model, num_heads, d_ff, dropout) 
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None):
        """
        Args:
            tgt (Tensor): Вход декодера (таргеты) [B, tgt_len, d_model]
            memory (Tensor): Выход энкодера [B, src_len, d_model]
            tgt_mask (Tensor, optional): Маска для self-attention в декодере [B, 1, tgt_len, tgt_len]
            memory_mask (Tensor, optional): Маска для cross-attention [B, 1, 1, src_len]
        Returns:
            Tensor: Выход декодера [B, tgt_len, d_model]
        """
        output = tgt
        for layer in self.layers:
            output = layer(output, memory, tgt_mask, memory_mask)
        return self.norm(output)


class TradingTransformer(nn.Module):
    """
    Полная модель Transformer для прогнозирования цен на фондовом рынке.
    """
    def __init__(
        self,
        feature_size=128,           # Размер признаков от TradingProcessor
        d_model=512,                # Размерность внутреннего представления
        num_encoder_layers=6,       # Количество слоев в энкодере
        num_decoder_layers=6,       # Количество слоев в декодере
        num_heads=8,                # Количество голов внимания
        d_ff=2048,                  # Размерность внутреннего слоя FFN
        target_len=32,              # Длина прогноза
        dropout=0.1,                # Dropout
        use_layer_norm=True,        # Использовать LayerNorm на входе
        max_seq_len=1000            # Максимальная длина последовательности для поз. кодирования
    ):
        """
        Args:
            feature_size (int): Размер входных признаков (128).
            d_model (int): Размерность внутреннего представления модели.
            num_encoder_layers (int): Количество слоев в энкодере.
            num_decoder_layers (int): Количество слоев в декодере.
            num_heads (int): Количество голов внимания.
            d_ff (int): Размерность внутреннего слоя FFN.
            target_len (int): Длина прогнозируемой последовательности (32).
            dropout (float): Вероятность Dropout.
            use_layer_norm (bool): Применять ли LayerNorm к входу.
            max_seq_len (int): Максимальная длина последовательности для поз. кодирования.
        """
        super(TradingTransformer, self).__init__()
        self.feature_size = feature_size
        self.d_model = d_model
        self.target_len = target_len
        self.use_layer_norm = use_layer_norm
        
        # Входные эмбеддинги и позиционное кодирование
        # Для истории
        self.src_embedding = nn.Linear(feature_size, d_model)
        # Для таргетов (предполагаем, что таргет это цена закрытия, 1 признак)
        self.tgt_embedding = nn.Linear(1, d_model)
        
        self.positional_encoding = PositionalEncoding(d_model, max_seq_len, dropout)
        
        # LayerNorm для нормализации входа (опционально)
        if use_layer_norm:
            self.src_layer_norm = nn.LayerNorm(feature_size)
            self.tgt_layer_norm = nn.LayerNorm(1) # Таргет это 1 признак
            
        # Энкодер и Декодер
        self.encoder = TransformerEncoder(
            num_layers=num_encoder_layers,
            d_model=d_model,
            num_heads=num_heads,
            d_ff=d_ff,
            dropout=dropout
        )
        
        self.decoder = TransformerDecoder(
            num_layers=num_decoder_layers,
            d_model=d_model,
            num_heads=num_heads,
            d_ff=d_ff,
            dropout=dropout
        )
        
        # Выходной слой для прогнозирования
        # Проектируем из d_model в размерность таргета (1)
        self.output_projection = nn.Linear(d_model, 1)
        
        # Инициализация весов
        self._init_weights()
        
    def _init_weights(self):
        """Инициализация весов."""
        for name, param in self.named_parameters():
            if 'weight' in name and len(param.shape) > 1:
                nn.init.xavier_uniform_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)

    def generate_square_subsequent_mask(self, sz):
        """
        Генерирует квадратную маску для предотвращения заглядывания в будущее.
        Args:
            sz (int): Размер квадратной матрицы.
        Returns:
            Tensor: Маска [sz, sz] с 0 для запрещенных позиций и -inf для разрешенных.
        """
        mask = torch.triu(torch.ones(sz, sz), diagonal=1)
        mask = mask.masked_fill(mask == 1, float('-inf'))
        return mask

    def _prepare_encoder_input(self, src):
        """Подготовка входа для энкодера."""
        batch_size, src_seq_len, _ = src.size()
        
        # Применяем Layer Normalization (если включено)
        if self.use_layer_norm:
            src_norm = self.src_layer_norm(src) # [B, src_seq_len, 128]
        else:
            src_norm = src
            
        # Линейное преобразование в d_model
        src_embedded = self.src_embedding(src_norm) # [B, src_seq_len, d_model]
        
        # Добавляем позиционное кодирование
        src_embedded = self.positional_encoding(src_embedded) # [B, src_seq_len, d_model]
        
        return src_embedded

    def _prepare_decoder_input(self, tgt):
        """Подготовка входа для декодера."""
        batch_size, tgt_seq_len, _ = tgt.size()
        
        # Применяем Layer Normalization к таргету (если включено)
        if self.use_layer_norm:
            tgt_norm = self.tgt_layer_norm(tgt) # [B, tgt_seq_len, 1]
        else:
            tgt_norm = tgt
            
        # Линейное преобразование в d_model
        tgt_embedded = self.tgt_embedding(tgt_norm) # [B, tgt_seq_len, d_model]
        
        # Добавляем позиционное кодирование
        tgt_embedded = self.positional_encoding(tgt_embedded) # [B, tgt_seq_len, d_model]
        
        return tgt_embedded

    def forward(self, src, tgt=None):
        """
        Прямой проход модели.
        
        Args:
            src (torch.Tensor): Исторические данные [B, T_hist=256, F_hist=128].
                                Должны быть уже обработаны TradingProcessor.
            tgt (torch.Tensor, optional): Целевые значения [B, T_pred=32, 1].
            
        Returns:
            torch.Tensor: Прогнозы [B, T_pred=32, 1].
        """
        # src: [B, 256, 128]
        # tgt: [B, 32, 1] (если предоставлено)
        
        # --- Обработка входа (энкодер) ---
        src_embedded = self._prepare_encoder_input(src)
        
        # Пропускаем через энкодер
        # memory: [B, 256, d_model]
        memory = self.encoder(src_embedded)
        
        # --- Обработка таргета (декодер) ---
        if self.training or tgt is not None:
            # Во время обучения используем предоставленные таргеты (teacher forcing)
            if tgt is None:
                raise ValueError("tgt must be provided during training")
                
            tgt_input = tgt # [B, 32, 1]
            tgt_embedded = self._prepare_decoder_input(tgt_input)
            
            # Генерируем маску для предотвращения заглядывания в будущее
            tgt_seq_len = tgt_embedded.size(1)
            tgt_mask = self.generate_square_subsequent_mask(tgt_seq_len).to(tgt_embedded.device)
            # tgt_mask: [tgt_seq_len, tgt_seq_len] -> [1, 1, tgt_seq_len, tgt_seq_len]
            tgt_mask = tgt_mask.unsqueeze(0).unsqueeze(0)
            
            # Пропускаем через декодер
            # decoder_output: [B, 32, d_model]
            decoder_output = self.decoder(
                tgt=tgt_embedded,
                memory=memory,
                tgt_mask=tgt_mask
            )
            
            # Применяем выходную проекцию
            # output: [B, 32, d_model] -> [B, 32, 1]
            output = self.output_projection(decoder_output)
            
        else:
            # Во время инференса генерируем таргеты авторегрессивно
            output = self.generate(src, memory)
            
        return output

    def generate(self, src, memory=None):
        """
        Авторегрессивная генерация прогноза.
        
        Args:
            src (torch.Tensor): Исторические данные [B, T_hist, F_hist].
            memory (torch.Tensor, optional): Предвычисленный выход энкодера [B, T_hist, d_model].
            
        Returns:
            torch.Tensor: Прогнозы [B, T_pred, 1].
        """
        batch_size = src.size(0)
        device = src.device
        
        if memory is None:
            # Вычисляем memory, если оно не предоставлено
            src_embedded = self._prepare_encoder_input(src)
            memory = self.encoder(src_embedded)
        
        # Начинаем с токена начала последовательности или последнего известного значения
        # Для простоты начнем с нулей
        # tgt: [B, 1, 1] - начинаем с одного токена
        tgt = torch.zeros(batch_size, 1, 1, device=device)
        
        # Список для хранения прогнозов
        predictions = []
        
        for _ in range(self.target_len):
            # Подготавливаем текущий tgt для декодера
            tgt_embedded = self._prepare_decoder_input(tgt) # [B, текущая_длина, d_model]
            
            # Генерируем маску для текущей длины
            current_tgt_len = tgt_embedded.size(1)
            tgt_mask = self.generate_square_subsequent_mask(current_tgt_len).to(device)
            tgt_mask = tgt_mask.unsqueeze(0).unsqueeze(0) # [1, 1, текущая_длина, текущая_длина]
            
            # Пропускаем через декодер
            # decoder_output: [B, текущая_длина, d_model]
            decoder_output = self.decoder(
                tgt=tgt_embedded,
                memory=memory,
                tgt_mask=tgt_mask
            )
            
            # Берем выход последнего токена (наш прогноз)
            # last_output: [B, d_model]
            last_output = decoder_output[:, -1, :]
            
            # Применяем выходную проекцию
            # next_prediction: [B, 1]
            next_prediction = self.output_projection(last_output)
            predictions.append(next_prediction.unsqueeze(1)) # [B, 1, 1]
            
            # Обновляем tgt для следующей итерации
            # Добавляем прогноз к tgt
            # tgt: [B, текущая_длина, 1]
            # next_prediction.unsqueeze(1): [B, 1, 1]
            tgt = torch.cat([tgt, next_prediction.unsqueeze(1)], dim=1) # [B, текущая_длина+1, 1]
            
        # Конкатенируем все прогнозы
        # Каждый элемент predictions: [B, 1, 1]
        # final_output: [B, T_pred, 1]
        final_output = torch.cat(predictions, dim=1)
        
        return final_output



# --- Пример использования ---
if __name__ == "__main__":
    # Параметры модели
    B, T_hist, feature_size = 4, 256, 128 # Уменьшено B для теста
    T_pred = 32
    output_size = 1
    
    # Создание модели
    model = TradingTransformer(
        feature_size=feature_size,
        d_model=512, 
        num_encoder_layers=6,
        num_decoder_layers=6,
        num_heads=8, 
        d_ff=2048, 
        target_len=T_pred,
        dropout=0.1,
        use_layer_norm=True,
        max_seq_len=1000
    )
    
    # Примерные входные данные
    src = torch.randn(B, T_hist, feature_size)  # История после TradingProcessor
    tgt = torch.randn(B, T_pred, output_size)   # Целевые значения
    
    # # Обучение (с учителем)
    # model.train()
    # output_train = model(src, tgt)
    # print(f"Выход при обучении (с учителем): {output_train.shape}") # [B, 32, 1]
    
    # Инференс
    model.eval()
    with torch.no_grad():
        output_infer = model(src)
        print(f"Выход при инференсе (авторегрессивно): {output_infer.shape}") # [B, 32, 1]


Выход при инференсе (авторегрессивно): torch.Size([4, 32, 1])


### Тестированиe TradingModel

In [1]:
from models.model_wrapper import TradingModel

# model = TradingModel.from_config('src/lstm/lstm.json', device='mps')
# model = TradingModel.from_config('src/tcn/tcn.json', device='mps')
# model = TradingModel.from_config('src/cnn/cnn.json', device='mps')
model = TradingModel.from_config('src/transformer/transformer.json', device='mps')
print(model.device)
print(model.model)

mps
TradingTransformer(
  (src_embedding): Linear(in_features=128, out_features=512, bias=True)
  (tgt_embedding): Linear(in_features=1, out_features=512, bias=True)
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (src_layer_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (tgt_layer_norm): LayerNorm((1,), eps=1e-05, elementwise_affine=True)
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x EncoderBlock(
        (self_attn): MultiHeadAttention(
          (W_q): Linear(in_features=512, out_features=512, bias=True)
          (W_k): Linear(in_features=512, out_features=512, bias=True)
          (W_v): Linear(in_features=512, out_features=512, bias=True)
          (W_o): Linear(in_features=512, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ffn): PositionwiseFeedForward(
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (